## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-23-21-50-00 +0000,wsj,Opinion | RFK Jr. Turns the CDC Against Vaccines,https://www.wsj.com/opinion/rfk-jr-turns-the-c...
1,2025-11-23-21-49-00 +0000,wsj,Opinion | The Meaning of Marjorie Taylor Greene,https://www.wsj.com/opinion/the-meaning-of-mar...
2,2025-11-23-21-47-15 +0000,wapo,Ukraine seeks urgent changes to Trump peace pl...,https://www.washingtonpost.com/world/2025/11/2...
3,2025-11-23-21-47-00 +0000,wsj,Opinion | Another Week in the Wild West Bank,https://www.wsj.com/opinion/another-week-in-th...
4,2025-11-23-21-47-00 +0000,wsj,Opinion | Suspicious Drones Over Europe,https://www.wsj.com/opinion/suspicious-drones-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,27
10,ukraine,20
16,plan,17
15,peace,12
44,end,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
93,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...,117
26,2025-11-23-20-20-15 +0000,nyt,"Ukraine, U.S. Will Meet About Trump’s Peace Pl...",https://www.nytimes.com/2025/11/23/world/europ...,106
157,2025-11-23-03-22-00 +0000,wsj,Trump Says Ukraine Peace Plan Isn’t Final Afte...,https://www.wsj.com/politics/policy/trump-says...,91
2,2025-11-23-21-47-15 +0000,wapo,Ukraine seeks urgent changes to Trump peace pl...,https://www.washingtonpost.com/world/2025/11/2...,89
185,2025-11-23-00-00-00 +0000,wsj,Russia Stuck to Its Demands on Ukraine. Many A...,https://www.wsj.com/world/russia/russia-stuck-...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
93,117,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...
39,36,2025-11-23-19-31-39 +0000,bbc,Israel kills top Hezbollah official in first a...,https://www.bbc.com/news/articles/cn81j54xjx1o...
121,33,2025-11-23-10-58-29 +0000,wapo,Japan’s sacred sumo ring is off-limits to wome...,https://www.washingtonpost.com/world/2025/11/2...
100,32,2025-11-23-13-39-57 +0000,nypost,"Ukraine peace plan was authored by US, Rubio s...",https://nypost.com/2025/11/23/world-news/ukrai...
56,27,2025-11-23-17-46-20 +0000,nypost,NJ ‘MAGA mom’ reveals the moment she discovere...,https://nypost.com/2025/11/23/us-news/nj-maga-...
111,25,2025-11-23-12-00-00 +0000,wsj,While President Trump and New York Mayor-elect...,https://www.wsj.com/economy/everyone-is-talkin...
74,23,2025-11-23-16-41-26 +0000,nypost,Jasmine Crockett roasts Marjorie Taylor Greene...,https://nypost.com/2025/11/23/us-news/jasmine-...
169,23,2025-11-23-02-00-00 +0000,wsj,How CEOs got more comfortable with tariffs: We...,https://www.wsj.com/economy/trade/ceo-tariffs-...
17,22,2025-11-23-20-48-21 +0000,nypost,"Long Island’s largest zoo, which faced animal ...",https://nypost.com/2025/11/23/us-news/long-isl...
52,22,2025-11-23-18-07-23 +0000,nypost,"NYC house fire kills 95-year-old, critically i...",https://nypost.com/2025/11/23/us-news/nyc-hous...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
